In [50]:
import pandas as pd
import pickle

In [52]:
import pandas as pd
import pickle

class CustomerDataExtractor:
    def __init__(self, df_path, vip_customers_path):
        self.df_path = df_path
        self.vip_cust_path = vip_customers_path
        self.df = self.read_data()
        self.vip_ids = self.read_vips()

    def read_data(self):
        with open(self.df_path, "rb") as f:
            data = pickle.load(f)
        return pd.DataFrame(data)

    def read_vips(self):
        with open(self.vip_cust_path, "r") as text:
            return [int(line.strip()) for line in text.readlines()]

    def transform(self):
        self.df = self.df.rename(columns={"id": "customer_id", "name": "customer_name"})
        self.df["customer_id"] = self.df["customer_id"].astype(int)
        self.df["customer_name"] = self.df["customer_name"].astype(str)
        self.df["registration_date"] = pd.to_datetime(self.df["customer_id"])
        self.df["is_vip"] = self.df["customer_id"].isin(self.vip_ids)

        self.df["orders"] = self.df["orders"].apply(lambda x: x if isinstance(x, list) else [])
        self.df = self.df.explode("orders", ignore_index=True)
        self.df = self.df[self.df["orders"].notna() & (self.df["orders"] != {})]
        self.df = self.df.join(pd.json_normalize(self.df.pop("orders")))

        # Convert order_id only if numeric
        self.df["order_id"] = self.df["order_id"].apply(
            lambda x: int(x) if str(x).isdigit() else pd.NA
        ).astype("Int64")

        self.df["order_date"] = pd.to_datetime(self.df["order_date"], errors="coerce")

        # Category mapping
        category_map = {1: "Electronics", 2: "Apparel", 3: "Books", 4: "Home Goods"}

        def safe_int(val):
            try:
                return int(str(val).strip())
            except:
                return None

        # Extract fields from items
        def extract_first_valid(items, field, cast_func=None):
            if isinstance(items, list):
                for item in items:
                    if isinstance(item, dict) and field in item:
                        val = item[field]
                        if cast_func:
                            try:
                                return cast_func(val)
                            except:
                                continue
                        return val
            return None

        self.df["product_id"] = self.df["items"].apply(lambda items: extract_first_valid(items, "item_id", int))
        self.df["product_name"] = self.df["items"].apply(lambda items: extract_first_valid(items, "product_name", str))
        self.df["category"] = self.df["items"].apply(
            lambda items: category_map.get(safe_int(extract_first_valid(items, "category")), "Misc")
        )
        self.df["unit_price"] = self.df["items"].apply(
            lambda items: extract_first_valid(items, "price", lambda x: float(str(x).replace("$", "").strip()))
        )
        self.df["item_quantity"] = self.df["items"].apply(
            lambda items: extract_first_valid(items, "quantity", int)
        )

        # Total item price
        def compute_total_price(items):
            if not isinstance(items, list): return 0.0
            total = 0.0
            for item in items:
                if isinstance(item, dict):
                    try:
                        price = float(str(item.get("price", "")).replace("$", "").strip())
                        quantity = int(str(item.get("quantity", "")).strip())
                        total += price * quantity
                    except (ValueError, TypeError):
                        continue
            return total

        self.df["total_item_price"] = self.df["items"].apply(compute_total_price)

        # Order total value and percentage
        self.df["order_total_value"] = self.df["order_total_value"].apply(
            lambda x: float(x) if str(x).replace(".", "", 1).replace("-", "", 1).isdigit() else 0.0
        )

        total_sum = self.df["order_total_value"].sum()
        self.df["total_order_value_percentage"] = self.df["order_total_value"].apply(
            lambda x: (x / total_sum) * 100 if total_sum != 0 else 0.0
        )

        # Final type enforcement
        self.df = self.df[[
            "customer_id", "customer_name", "registration_date", "is_vip",
            "order_id", "order_date", "product_id", "product_name",
            "category", "unit_price", "item_quantity", "total_item_price",
            "total_order_value_percentage"
        ]]

        self.df = self.df.astype({
            "customer_id": int,
            "customer_name": str,
            "registration_date": "datetime64[ns]",
            "is_vip": bool,
            "order_id": "Int64",
            "order_date": "datetime64[ns]",
            "product_id": "Int64",
            "product_name": str,
            "category": str,
            "unit_price": float,
            "item_quantity": "Int64",
            "total_item_price": float,
            "total_order_value_percentage": float
        })

    def save_to_csv(self, output_path="cleaned_customer_data.csv"):
        self.df.to_csv(output_path, index=False)


In [54]:
pp = CustomerDataExtractor("customer_orders.pkl", "vip_customers.txt")

In [56]:
pp.transform()


In [58]:
pp.df

,customer_id,customer_name,registration_date,is_vip,order_id,order_date,product_id,product_name,category,unit_price,item_quantity,total_item_price,total_order_value_percentage
0,1,Customer 1,1970-01-01 00:00:00.000000001,True,84,2024-02-27 21:24:16,<NA>,None,Misc,NaN,<NA>,0.00,0.000000
1,1,Customer 1,1970-01-01 00:00:00.000000001,True,<NA>,2025-05-03 08:09:20,1,Item 1 for Order 2,Home Goods,52.70,2,2765.80,2.394045
2,1,Customer 1,1970-01-01 00:00:00.000000001,True,3,2023-09-06 00:42:50,1,Item 1 for Order 3,Electronics,377.96,1,377.96,0.000000
3,2,Customer 2,1970-01-01 00:00:00.000000002,False,3,2023-01-01 07:32:50,<NA>,None,Misc,NaN,<NA>,0.00,0.000000
4,2,Customer 2,1970-01-01 00:00:00.000000002,False,5,2024-08-28 19:37:56,1,Item 1 for Order 5,Misc,342.68,2,2245.37,-0.049482
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,49,Customer 49,1970-01-01 00:00:00.000000049,False,<NA>,NaT,<NA>,None,Misc,NaN,<NA>,0.00,0.000000
104,49,Customer 49,1970-01-01 00:00:00.000000049,False,<NA>,NaT,<NA>,None,Misc,NaN,<NA>,0.00,0.000000
105,50,Customer 50,1970-01-01 00:00:00.000000050,False,<NA>,NaT,<NA>,None,Misc,NaN,<NA>,0.00,0.000000
106,50,Customer 50,1970-01-01 00:00:00.000000050,False,<NA>,NaT,<NA>,None,Misc,NaN,<NA>,0.00,0.000000


In [60]:
pp.save_to_csv()